In [30]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys

import bs4
from bs4 import BeautifulSoup

import requests
import pandas as pd
import time
import numpy as np


In [2]:
#appending data frame to existing excel file
def append_df_to_excel(df, excel_path):
    df_excel = pd.read_excel(excel_path)
    result = pd.concat([df_excel, df], ignore_index=True)
    result.to_excel(excel_path, index=False)


In [3]:
# #reading provinces list
# read_province_list = open("jawa_bali.txt",'r')
# provinces=[]
# # # for _ in range(33): #skipping province
# # #     next(read_province_list)
# for province in read_province_list:
#     provinces.append(province.replace("\n", ", Indonesia").strip())

# # provinces[33]+=", Indonesia"
# # provinces = pd.read_excel(r"Indonesias_Hotel_List.xlsx",usecols=[2],skiprows=[0]).drop_duplicates().values.tolist()


# #provinces=[provinces[44]]
# print((provinces))

In [4]:
kota = pd.read_csv(r"Indonesian_Family_Life_Survey_4_Longitude_and_Latitude.csv", usecols=[5,6]).drop_duplicates()
len(kota)
for j in range(101,102):
    print(kota.iloc[j,1])

JAKARTA SELATAN


In [55]:
#url link
my_url='https://www.traveloka.com/en-id/'


#calling web driver from its directory
s=Service(r'C:\Users\62811\Downloads\geckodriver-v0.30.0-win64\geckodriver.exe') 


def strip(x):
    if x !=None:
        return   str(x.text.strip())
    else:
        return str('')

def strip_(x):
    if x !=None:
        return  x.text.strip()
    else:
        return ''



In [144]:
#url link
my_url='https://www.traveloka.com/en-id/'


#calling web driver from its directory
s=Service(r'C:\Users\62811\Downloads\geckodriver-v0.30.0-win64\geckodriver.exe') 

city='bandung'
driver = webdriver.Firefox(service=s)
wait = WebDriverWait(driver, 30)
driver.get(my_url)
search_box = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[data-testid='autocomplete-field']"))).send_keys(city)

(wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@data-testid='autocomplete-item-name' and contains(string(), %s)]"%city)))).click()

wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "[data-testid='search-submit-button']"))).click()



x=''
while True:
    try:
        time.sleep(4)
        x = x + wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "_2qd8A"))).get_attribute('innerHTML') #find_elements_by_class_name("_7192d3184")
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID,"next-button"))).click()
        print("Clicked on  Next Page »")
    except TimeoutException:
        print("No more Next Page »")
        break
driver.quit()



import winsound
duration = 1000  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)


Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
Clicked on  Next Page »
No more Next Page »


In [145]:
# x = open("card.html","r")
# card=BeautifulSoup(x, 'html.parser')
card=BeautifulSoup(x, 'html.parser').find_all(True,{'class':'ztzlF _2nyWi'})
# len(card)

#card sudah di simpan di C:\Users\62811\Documents\Web Scraping Project (traveloka.com)

In [151]:
i=0
list = [ ]
for item in card:

    name = strip(item.find(True, {'class':'_1z5je _10ZQX tvat-hotelName'}))
    
    loc = strip(item.find(True,{'class':'_3tICV'}))
    
    rating = strip(item.find(True,{'class':'tvat-ratingScore'}))

    rating_label_=''
    rating_label_=strip(item.find(True,{'class':'_16hDy'})).split()
    # rating_label=(type(rating_label_),len(rating_label_))

    # rating_label = strip(item.find(True,{'class':'_16hDy'})).split()
    if rating_label_  == []:
        rating_label = ''
    else:
        rating_label = rating_label_[0]
    # except:
    #     rating_label = ''

    number_of_reviews = ''
    number_of_reviews = strip(item.find(True,{'class':'_30os4'}))[1:-1]



    price = strip(item.find(True,{'class':'_22n9I tvat-primaryPrice'}))
    
    type = strip(item.find(True,{'class':'_3ohst Jewfo _2Vswb'}))
    
    # price_for= strip(item.find(True,{'class':'_4abc4c3d5 _7ee1c7d14'}))

    stars = item.find(True,{'class':'_1RoiH _1dIAz tvat-starRating _1Fq-V'})

    # city = kota.iloc[j,1]

    # province_name = kota.iloc[j,0]

    if stars == None:
        stars_count = ''
    elif len(stars) == 1:
        stars_count = ''
    else:
        stars_count=str(len(stars)-1)

    # href_tag= item.find('a', href=True)['href']

    list_in_list = [name,loc,rating,rating_label,number_of_reviews,price,type,stars_count]#,bed,price_for,stars_count,city,href_tag]
    
    list.append(list_in_list)
    #print(list_in_list)
    i=i+1
df = pd.DataFrame(list,columns= ['name','loc','rating','rating_label','number_of_reviews','price','type','stars_count'])
df.replace('', np.nan, inplace=True)
df.dropna(axis=0,how='all')

,name,loc,rating,rating_label,number_of_reviews,price,type,stars_count
0,eL Hotel Royale Bandung,"Merdeka, Bandung",8.7,Impressive,15577,Rp 900.000,Hotels,4
1,Grand Tjokro Premiere Bandung,"Cihampelas, Bandung",8.6,Impressive,15273,Rp 1.283.333,Hotels,4
2,The Trans Luxury Hotel,"Buahbatu, Bandung",9.0,Impressive,6178,Rp 3.302.750,Hotels,5
3,FOX Lite Hotel Metro Indah - Bandung,"Batununggal, Bandung",8.5,Impressive,4710,Rp 298.500,Hotels,3
4,De Batara Hotel,"Cihampelas, Bandung",8.2,Convenient,2978,Rp 289.900,Hotels,3
...,...,...,...,...,...,...,...,...
1231,Cozy Villa Mawar,"Mekarsaluyu, Bandung",NaN,NaN,NaN,Rp 1.500.000,Villas,NaN
1232,VILLA 5CEMARA,"Parongpong, Bandung",NaN,NaN,NaN,Rp 1.900.000,Villas,1
1233,ALI1 Pondok Orange 1 Ciwidey,"Ciwidey, Bandung",8.8,Impressive,84,Rp 694.445,Homestays,1
1234,La Fasa Syariah Hotel,"Jatinangor, Bandung",8.1,Convenient,239,Rp 850.000,Hotels,1


In [88]:
rating_label_=strip(card.find(True,{'class':'_16hDy'})).split()
rating_label_[0]

'Impressive'

In [127]:
f = open("card.html", "a")
f.write(str(card))
f.close()

In [10]:
r = requests.get("https://www.traveloka.com/en-id/")

page = BeautifulSoup(r.text,'html.parser')

driver.find_elements(by=By.CLASS_NAME, value="_7192d3184")

review = [x for x in page.find_all(True, {'class':'css-1dbjc4n r-1awozwy r-18u37iz r-1777fci r-edyy15'})]
print(review)

[<div class="css-1dbjc4n r-1awozwy r-18u37iz r-1777fci r-edyy15"><div class="css-1dbjc4n r-kdyh1x r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af r-14qw7yl r-1wyyakw" style="background-color:rgba(255,255,255,1.00);border-top-color:rgba(205,208,209,1.00);border-right-color:rgba(205,208,209,1.00);border-bottom-color:rgba(205,208,209,1.00);border-left-color:rgba(205,208,209,1.00);border-top-width:1px;border-right-width:1px;border-bottom-width:1px;border-left-width:1px;height:calc(100% - 2);-webkit-transition-delay:0ms;transition-delay:0ms;-webkit-transition-property:width, height, border-color, border-width, background-color;transition-property:width, height, border-color, border-width, background-color;-webkit-transition-timing-function:ease;transition-timing-function:ease;width:calc(100% - 2);will-change:width, height, border-color, border-width, background-color;-moz-transition-property:width, height, border-color, border-width, background-color"></div><div class="css-1dbjc4n r-1kb76zh">

In [8]:
len(review)

2